# Linear model: Elastic Net

Reviewer 1 asked me to make comparison with simple linear model.

In [2]:
import pickle

with open('../../PNC_Good/MegaMeta3.pkl', 'rb') as f: 
    meta = pickle.load(f)
    
print('Complete')

Complete


In [3]:
with open('../../Work/LatentSim/Splits.pkl', 'rb') as f:
    keys, groups = pickle.load(f)
    
print(len(keys))
print(len(groups[0][0])+len(groups[0][1]))
print('Complete')

620
620
Complete


In [4]:
import numpy as np

nback = np.stack([meta[key]['nback'] for key in keys])
emoid = np.stack([meta[key]['emoid'] for key in keys])

print(nback.shape)
print(emoid.shape)

(620, 264, 231)
(620, 264, 210)


In [5]:
from nilearn.connectome import ConnectivityMeasure

def getFC(timeSeries, kind='correlation', transpose=True):
    connMeasure = ConnectivityMeasure(kind=kind)
    if transpose:
        timeSeries = np.transpose(timeSeries, axes=(0,2,1))
    return connMeasure.fit_transform(timeSeries)

nback_p = getFC(nback)
emoid_p = getFC(emoid)

print(nback_p.shape)

(620, 264, 264)


In [6]:
import torch

nback_pp_t = torch.from_numpy(nback_p).float().cuda()
emoid_pp_t = torch.from_numpy(emoid_p).float().cuda()

print(emoid_pp_t.shape)

torch.Size([620, 264, 264])


In [7]:
import torch

def convertTorch(p):
    t = torch.from_numpy(p).float()
    u = []
    for i in range(t.shape[0]):
        u.append(t[i][torch.triu_indices(264,264,offset=1).unbind()])
    return torch.stack(u).cuda()

nback_p_t = convertTorch(nback_p)
emoid_p_t = convertTorch(emoid_p)

print(nback_p_t.shape)
print(emoid_p_t.shape)
print('Complete')

torch.Size([620, 34716])
torch.Size([620, 34716])
Complete


In [8]:
mu_nback = torch.mean(nback_p_t, dim=0, keepdim=True)
mu_emoid = torch.mean(emoid_p_t, dim=0, keepdim=True)
std_nback = torch.std(nback_p_t, dim=0, keepdim=True)
std_emoid = torch.std(emoid_p_t, dim=0, keepdim=True)

nback_p_t = (nback_p_t - mu_nback)/std_nback
emoid_p_t = (emoid_p_t - mu_emoid)/std_emoid

print('Norm complete')

Norm complete


In [9]:
print(torch.min(nback_p_t[:,420:422],dim=0))
print(torch.min(emoid_p_t[:,420:422],dim=0))

torch.return_types.min(
values=tensor([-2.2998, -3.0133], device='cuda:0'),
indices=tensor([603, 350], device='cuda:0'))
torch.return_types.min(
values=tensor([-2.7284, -3.0616], device='cuda:0'),
indices=tensor([351,  33], device='cuda:0'))


In [10]:
age = np.stack([meta[key]['AgeInMonths'] for key in keys])
gen = np.stack([np.array([meta[key]['Gender'] == 'M', meta[key]['Gender'] == 'F']) for key in keys]).astype(int)
wrt = np.stack([meta[key]['wratStd'] for key in keys])

print(age.shape)
print(gen.shape)
print(wrt.shape)

(620,)
(620, 2)
(620,)


In [11]:
age_t = torch.from_numpy(age).float().cuda()
gen_t = torch.from_numpy(gen).float().cuda()
wrt_t = torch.from_numpy(wrt).float().cuda()

print('Complete')

Complete


In [12]:
import sys

sys.path.append('../../LatentSimilarity')

from latsim import LatSim

print('Complete')

Complete


In [30]:
import torch
import torch.nn as nn

mseLoss = nn.MSELoss()
l1loss = 0
verbose = True
nepochs = 100
pperiod = 50
allres = []

for mod in [nback_p_t, emoid_p_t]:
    trainIdcs = groups[0][0][0:496]
    testIdcs = groups[0][1]
    x = nback_p_t.unsqueeze(1)
    xtr = nback_p_t[trainIdcs].unsqueeze(1)
    xt = nback_p_t[testIdcs].unsqueeze(1)
    ytr = wrt_t[trainIdcs]
    yt = wrt_t[testIdcs]
    
    sim = LatSim(1, xtr, dp=0.5, edp=0.1, wInit=1e-4, dim=2, temp=1)
    optim = torch.optim.Adam(sim.parameters(), lr=1e-4, weight_decay=1e-4)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)

    for epoch in range(nepochs):
        optim.zero_grad()
        yhat = sim(xtr, [ytr])[0][0]
        loss = mseLoss(yhat, ytr)
        loss.backward()
        optim.step()
        sched.step(loss)
        if verbose:
            if epoch % pperiod == 0 or epoch == nepochs-1:
                print(f'{epoch} {[float(l) for l in [loss, l1loss]]} {sched._last_lr}')
    
    res = sim(x, [wrt_t], testIdcs)[0][0][testIdcs]
    ls = mseLoss(res, yt)**0.5
    print(ls)
    allres.append(res.detach())
    
res = sum(allres)/2
ls = mseLoss(res, yt)**0.5
print(ls)
print(np.corrcoef(res.detach().cpu().numpy(), yt.detach().cpu().numpy()))

0 [266.8812255859375, 0.0] [0.0001]
50 [14.992171287536621, 0.0] [0.0001]
99 [9.056427955627441, 0.0] [7.500000000000001e-05]
tensor(13.7968, device='cuda:0', grad_fn=<PowBackward0>)
0 [267.9695739746094, 0.0] [0.0001]
50 [11.86994743347168, 0.0] [0.0001]
99 [8.851895332336426, 0.0] [7.500000000000001e-05]
tensor(13.7186, device='cuda:0', grad_fn=<PowBackward0>)
tensor(13.5361, device='cuda:0')
[[1.         0.42348294]
 [0.42348294 1.        ]]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

mseLoss = nn.MSELoss()
ceLoss = nn.CrossEntropyLoss()

class ElasticNet(nn.Module):
    def __init__(self, nfeat, nout=1):
        super(ElasticNet, self).__init__()
        self.l1 = nn.Linear(nfeat, nout).float().cuda()
        self.cls = nout > 1
        
    def train(self, xtr, ytr, nepochs=1000, lr=1e-1, l1=1e-1, l2=1e-4, pperiod=100, verbose=False):
        optim = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=l2)
        sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)
        
        for epoch in range(nepochs):
            optim.zero_grad()
            yhat = self(xtr)
            if self.cls:
                loss = ceLoss(yhat, ytr)
            else:
                loss = mseLoss(yhat, ytr)**0.5
            l1loss = l1*torch.sum(torch.abs(self.l1.weight))
            (loss+l1loss).backward()
            optim.step()
            sched.step(loss)
            if verbose:
                if epoch % pperiod == 0 or epoch == nepochs-1:
                    print(f'{epoch} {[float(l) for l in [loss, l1loss]]} {sched._last_lr}')
                    
    def predict(self, xt, yt):
        with torch.no_grad():
            if self.cls:
                return torch.argmax(self(xt), dim=-1)
            else:
                return mseLoss(self(xt), yt)**0.5
                    
    def forward(self, x):
        return self.l1(x).squeeze()
    
print('Complete')

In [48]:
len(groups)

10

In [91]:
losses = []
import time
t0 = time.time()
for group in range(10):
    trainIdcs = groups[group][0][:496]
    valIdcs = groups[group][0][496:]
    testIdcs = groups[group][1]
    
    xtr = nback_p_t[trainIdcs]
    xtr = torch.cat([xtr, torch.ones(xtr.shape[0],1).float().cuda()], dim=1)
    xt = nback_p_t[testIdcs]
    xt = torch.cat([xt, torch.ones(xt.shape[0],1).float().cuda()], dim=1)
    ytr = gen_t[trainIdcs]
    yt = gen_t[testIdcs]
    
    w, _, _, _ = torch.linalg.lstsq(xtr, ytr)
#     print(mseLoss(xt@w, yt)**0.5)
    nb = xt@w
    
    net = ElasticNet(xtr.shape[-1],2)
    net.train(xtr, ytr, lr=1e-1, l1=1e-2, l2=1e-2, nepochs=2000, pperiod=400, verbose=True)
    nb = net(xt)
    
    xtr = emoid_p_t[trainIdcs]
    xtr = torch.cat([xtr, torch.ones(xtr.shape[0],1).float().cuda()], dim=1)
    xt = emoid_p_t[testIdcs]
    xt = torch.cat([xt, torch.ones(xt.shape[0],1).float().cuda()], dim=1)
    
    w, _, _, _ = torch.linalg.lstsq(xtr, ytr)
#     print(mseLoss(xt@w, yt)**0.5)
    em = xt@w
    
    net = ElasticNet(xtr.shape[-1],2)
    net.train(xtr, ytr, lr=1e-1, l1=1e-2, l2=1e-2, nepochs=2000, pperiod=400, verbose=True)
    em = net(xt)
    
    res = (nb+em)/2
    res = torch.argmax(res, dim=1)
    loss = torch.sum(res == torch.argmax(yt, dim=1))/res.shape[0]
#     loss = mseLoss(res, yt)**0.5
    print(loss)
    losses.append(loss)

for loss in losses:
    print(float(loss))
t1=time.time()

0 [0.7008514404296875, 1.8692128658294678] [0.1]
400 [0.3171605169773102, 0.6713947057723999] [1.004524257206333e-05]
800 [0.31738588213920593, 0.6699857711791992] [3.181966398799364e-07]
1200 [0.31748315691947937, 0.6694215536117554] [3.181966398799364e-07]
1600 [0.31761300563812256, 0.6686912178993225] [3.181966398799364e-07]
1999 [0.31777819991111755, 0.6677947044372559] [3.181966398799364e-07]
0 [0.7010551691055298, 1.8515889644622803] [0.1]
400 [0.33742472529411316, 0.4732629954814911] [1.004524257206333e-05]
800 [0.3376302421092987, 0.47237861156463623] [3.181966398799364e-07]
1200 [0.33771565556526184, 0.47206735610961914] [3.181966398799364e-07]
1600 [0.33782798051834106, 0.47166094183921814] [3.181966398799364e-07]
1999 [0.33796876668930054, 0.47115612030029297] [3.181966398799364e-07]
tensor(0.8387, device='cuda:0')
0 [0.6992988586425781, 1.8612139225006104] [0.1]
400 [0.3276269733905792, 0.6196433305740356] [5.650448946785623e-06]
800 [0.3277336657047272, 0.6187986135482788]

1999 [0.3445482552051544, 0.42699745297431946] [3.181966398799364e-07]
tensor(0.7258, device='cuda:0')
0.8387096524238586
0.7096773982048035
0.8225806355476379
0.7419354915618896
0.8064515590667725
0.7903225421905518
0.7419354915618896
0.7903225421905518
0.8387096524238586
0.7258064150810242


In [93]:
print((t1-t0)/3)

11.335182428359985
